# 실행하기 전에
- content에와 같은 경로에 test폴더 생성 후, test 폴더 안에 images, masks 폴더 생성

```
content
test
  ㄴ images
  ㄴ masks
```



# Data load and patch

In [ ]:
!git clone https://github.com/louisyuzhe/car-damage-detector.git

In [ ]:
# !git clone https://github.com/nyknstyn/car-damage-detect.git

In [ ]:
# !wget https://github.com/Rajat-Roy/car_damage/releases/download/v1.0/dataset.zip
# !unzip dataset.zip

In [3]:
cd /content/car-damage-detector

/content/car-damage-detector


In [ ]:
# !pip install -r requirements.txt

In [ ]:
!pip install tensorflow==1.13.1
!pip install keras==2.0.8
!pip install h5py==2.10.0

# Import

In [ ]:
#Import modules
import os
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon

import skimage.draw

import shutil

# Import Mask RCNN
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
# from mrcnn import model
# import mrcnn.model as modellib
# from mrcnn.model import log
import cv2
import custom
import imgaug,h5py,IPython

%matplotlib inline

In [ ]:
# annotations1 = json.load(open(os.path.join(data_path + '/' + "via_region_data.json"),'r',encoding="utf8",errors='ignore'))
# annotations = list(annotations1.values())
# annotations = list(annotations[1].values())
# annotations = [a for a in annotations if a['regions']]   

# Root directory of the project

In [16]:
# Root directory of the project
ROOT_DIR = os.getcwd()
sys.path.append(ROOT_DIR)  # To find local version of the library

config = custom.CustomConfig()
dataset_DIR = os.path.join(ROOT_DIR, "")
dataset_DIR

'/content/car-damage-detector/'

In [ ]:
dataset = custom.CustomDataset()
dataset.load_custom(dataset_DIR, "train")
dataset.load_custom(dataset_DIR, "val")
dataset.prepare()

print("Image Count: {}".format(len(dataset.image_ids)))
print("Class Count: {}".format(dataset.num_classes))
for i, info in enumerate(dataset.class_info):
    print("{:3}. {:50}".format(i, info['name']))

# CustomDataset

In [13]:
# -*- coding: utf-8 -*-
"""
Created on Tue Dec 15 00:05:33 2020

@author: Yu Zhe
"""
    
class CustomDataset(utils.Dataset):
    def load_custom(self, dataset_dir, subset):
        """Load a subset of the dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("scratch", 1, "scratch")
        
        # Train or validation dataset?
        assert subset in ["train"]
        dataset_dir = os.path.join(dataset_dir + "dataset/" + subset)
      
        # Load annotations
        # VGG Image Annotator saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }

        annotations1 = json.load(open(os.path.join(dataset_dir + '/' + "via_region_data.json"),'r',encoding="utf8",errors='ignore'))
        annotations = list(annotations1.values())
        # annotations = list(annotations[1].values()) # regions shape에 따라 주석처리 필요
        annotations = [a for a in annotations if a['regions']]   
        
        # Add images
        for a in annotations:
            polygons = []
            for i in range(len(a['regions'])):
                polygons.append(a['regions'][i]['shape_attributes'])
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "scratch",  ## for a single class just add the name here
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons)

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a balloon dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "scratch":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "scratch":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

## Test

In [ ]:
        # annotations1 = json.load(open(os.path.join(dataset_dir + '/' + "via_region_data.json"),'r',encoding="utf8",errors='ignore'))
        # annotations = list(annotations1.values())
        # annotations = list(annotations[1].values())
        # annotations = [a for a in annotations if a['regions']]   
        
        # # Add images
        # for a in annotations:
        #     print(a)
        #     # Get the x, y coordinaets of points of the polygons that make up
        #     # the outline of each object instance. There are stores in the
        #     # shape_attributes (see json format above)
        #     polygons = []
        #     for i in range(len(a['regions'])):
        #         print(a['regions'][i]['shape_attributes'])
        #         polygons.append(a['regions'][i]['shape_attributes'])

In [ ]:
data_path = '/content/car-damage-detector/dataset/train'
annotations1 = json.load(open(os.path.join(data_path + '/' + "via_region_data.json"),'r',encoding="utf8",errors='ignore'))
print(annotations1)
annotations = list(annotations1.values())
print('3 =>', annotations[3])
# annotations = list(annotations[1].values())
# print(annotations)
annotations = [a for a in annotations if a['regions']]
print(annotations)

for a in annotations:
    # print(a)
    # print(len(a['regions']))
    polygons = []
    objects = []
    # polygons = [r['shape_attributes'] for r in a['regions']]
    # objects = [s['region_attributes'] for s in a['regions']]
    # print(polygons)
    # print(objects)
    for i in range(len(a['regions'])):
        print(a['regions'][i]['region_attributes'])
        objects.append(a['regions'][i]['region_attributes'])
        # print(a['regions'][i]['shape_attributes'])
        polygons.append(a['regions'][i]['shape_attributes'])

    # polygons = [r['shape_attributes'] for r in a['regions'].values()]
    # objects = [s['region_attributes'] for s in a['regions']]
    # print('-----polygons: ', polygons)
    # print('-----objects: ', objects)
    print(objects)

# Data check

In [ ]:
# Load dataset
dataset = CustomDataset()
dataset.load_custom(dataset_DIR, "train")
dataset.load_custom(dataset_DIR, "val")
dataset.prepare()

print("Image Count: {}".format(len(dataset.image_ids)))
print("Class Count: {}".format(dataset.num_classes))
for i, info in enumerate(dataset.class_info):
    print("{:3}. {:50}".format(i, info['name']))

In [ ]:
# dataset.image_info

# Display function

In [ ]:
def display_images(images, titles=None, cols=5, cmap=None, norm=None,
                   interpolation=None):
    """Display the given set of images, optionally with titles.
    images: list or array of image tensors in HWC format.
    titles: optional. A list of titles to display with each image.
    cols: number of images per row
    cmap: Optional. Color map to use. For example, "Blues".
    norm: Optional. A Normalize instance to map values to colors.
    interpolation: Optional. Image interpolation to use for display.
    """
    titles = titles if titles is not None else [""] * len(images)
    rows = len(images) // cols + 1
    plt.figure(figsize=(14, 14 * rows // cols))
    i = 1
    for image, title in zip(images, titles):
        plt.subplot(rows, cols, i)
        plt.title(title, fontsize=9)
        plt.axis('off')
        plt.imshow(image.astype(np.uint8), cmap=cmap,
                   norm=norm, interpolation=interpolation)
        i += 1
    plt.show()

In [ ]:
def display_top_masks(image, mask, class_ids, class_names, limit=5):
    """Display the given image and the top few class masks."""
    to_display = []
    titles = []
    to_display.append(image)
    titles.append("H x W={}x{}".format(image.shape[0], image.shape[1]))
    # Pick top prominent classes in this image
    unique_class_ids = np.unique(class_ids)
    mask_area = [np.sum(mask[:, :, np.where(class_ids == i)[0]])
                 for i in unique_class_ids]
    top_ids = [v[0] for v in sorted(zip(unique_class_ids, mask_area),
                                    key=lambda r: r[1], reverse=True) if v[1] > 0]
    # Generate images and titles
    for i in range(limit):
        class_id = top_ids[i] if i < len(top_ids) else -1
        # Pull masks of instances belonging to the same class.
        m = mask[:, :, np.where(class_ids == class_id)[0]]
        m = np.sum(m * np.arange(1, m.shape[-1] + 1), -1)
        to_display.append(m)
        titles.append(class_names[class_id] if class_id != -1 else "-")
    display_images(to_display, titles=None, cols=limit + 1, cmap="Blues_r")

    return to_display, titles

In [ ]:
dataset.load_mask(dataset.image_ids[0])

# Image check and save

In [ ]:
# img_path = '/content/drive/MyDrive/Colab Notebooks/alpaco/Team_1. CODEnter [ Save Form : Name_FileName_Date ]/1. MAIN/6. Data/a_new_data_0627/train/'

In [ ]:
img_path = '/content/test/'

In [ ]:
image_ids = dataset.image_ids
file_name_list = []
file_name_only = []
for image_id in image_ids:
    file_names = dataset.image_info[image_id]['id']
    file_name_list.append(file_names)
    file_name = file_names.split('.')[0]
    file_name_only.append(file_name)
    print(file_name)
    
    image = dataset.load_image(image_id)
    mask, class_ids = dataset.load_mask(image_id)
    a, b = display_top_masks(image, mask, class_ids, dataset.class_names)

    # plt.imsave(img_path+f'mask/b_{file_name}.jpg', a[0], cmap = 'gray')

# Image move

In [ ]:
before_path = '/content/car-damage-detector/dataset/train/'
move_path = '/content/test/images/'

In [ ]:
for name, only in zip(file_name_list, file_name_only):
    # print(test_path + name)
    shutil.move(before_path + name, move_path+f'/b_{only}.jpg')

# Data zip

In [ ]:
cd /content/test

/content


In [ ]:
!zip -r b_images.zip ./images
!zip -r b_mask.zip ./mask

# Practice

In [ ]:
# a_path = '/content/car-damage-detector/dataset/train'
# b_path = '/content/car-damage-detector/dataset/val'

In [ ]:
# a_dir = os.listdir(a_path)
# b_dir = os.listdir(b_path)

In [ ]:
# all_dir = []
# all_path = []

In [ ]:
# all_dir

In [ ]:
# for dir in a_dir:
#     if dir[-4:] != 'json':
#         all_dir.append(dir.split('.')[0])
#         all_path.append(a_path+'/'+dir)

In [ ]:
# for dir in b_dir:
#     if dir[-4:] != 'json':
#         all_dir.append(dir.split('.')[0])
#         all_path.append(b_path+'/'+dir)

In [ ]:
# for a, b in zip(all_path, all_dir):
#     print(a, b)

In [ ]:
# import shutil

In [ ]:
# for path, dir in zip(all_path, all_dir):
#     shutil.move(path, img_path+f'images/{dir}.jpg')

In [ ]:
# img_paths = '/content/drive/MyDrive/Colab Notebooks/alpaco/Team_1. CODEnter [ Save Form : Name_FileName_Date ]/1. MAIN/6. Data/a_new_data_0627/train/masks'

In [ ]:
# a_dir = os.listdir(img_paths)
# a_dir

In [ ]:
# for a, all in zip(a_dir, all_dir):
#     img_path = img_paths + '/' + a
#     shutil.move(img_path, img_paths+f'/a_{all}.jpg')